In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Define surveys

In [ ]:
dragons    = {'file':'dragons',   'name':'DRAGONS',   'fmin':350e6,  'fmax':1030e6, 'df':1e6}
chime_full = {'file':'chime_full','name':'CHIME ',    'fmin':400e6,  'fmax':800e6,  'df':0.4e6}
chime_low  = {'file':'chime_low', 'name':'CHIME low', 'fmin':400e6,  'fmax':440e6,  'df':0.4e6}
chime_high = {'file':'chime_high','name':'CHIME high','fmin':600e6,  'fmax':800e6,  'df':0.4e6}
gmims_hbn  = {'file':'gmims_hbn', 'name':'GMIMS HBN', 'fmin':1280e6, 'fmax':1750e6, 'df':1e6}
gmims_lbs  = {'file':'gmims_lbs', 'name':'GMIMS LBS', 'fmin':300e6,  'fmax':480e6,  'df':0.5e6}
#staps      = {'file':, 'name':,'fmin': , 'fmax': ,  'df':}
#pegasus    = {'file':, 'name':, fmin': , 'fmax': ,  'df':}

## Pick the surveys to plot, and FD range and sampling

In [ ]:
phi_max    = 200 # range of FD values to plot for RMSF (+/-)
dphi       = 0.5 # FD sampling
survey_all = [dragons,chime_full] # which surveys to plot (1 or more)
colors     = ['C2','C1'] # colors to use for plotting each survey 
combine    = False # Set True if want to combine frequency coverage from chosen surveys
file_out   = 'dragons_chime_all.jpg' # name of the file to save

## Some functions

In [ ]:
c = 2.998e8
freq = np.asarray(np.linspace(200,2000,2001))
lbd2 = (c/(freq*1e6))**2

def get_survey_params(survey):
    
    phi = []
    rmsf= []
    with open(survey['file']+'.txt', 'r') as file:
        for line in file:
            phi.append(float(line.split()[0]))
            rmsf.append(float(line.split()[1]))

    with open(survey['file']+'_params.txt', 'r') as file:
        for line in file:
            if line.split()[0] == 'phi_broad':
                phi_broad = float(line.split()[1])
    
    return phi, rmsf, phi_broad

def get_combo_survey_params():
    
    phi = []
    rmsf= []
    with open('combo.txt', 'r') as file:
        for line in file:
            phi.append(float(line.split()[0]))
            rmsf.append(float(line.split()[1]))

    with open('combo_params.txt', 'r') as file:
        for line in file:
            if line.split()[0] == 'phi_broad':
                phi_broad = float(line.split()[1])
    
    return phi, rmsf, phi_broad

def make_the_plot(survey_all,colors,freqs_all,phi_broad_all,phi_all,rmsf_all,
                  phi_combo=None, rmsf_combo=None, phi_broad_combo=None, do_combo=False,
                  fs=24,save_plot=True,file_out='test.jpg',*args,**kwargs):
    
    fig, ax = plt.subplots(figsize=(25,8))

    # Make the coverage plot:

    ax.plot(lbd2,freq,color='black',zorder=100)
    for j in range(0,len(freqs_all)):
        freqs = freqs_all[j]
        y = np.arange(freqs[0],freqs[1],0.1*1e6)
        x = (c/y)**2
        x1 = (np.arange(0,c/freqs[0],0.001))**2
        x2 = (np.arange(0,c/freqs[1],0.001))**2
        y1 = np.repeat(freqs[0],len(x1))
        y2 = np.repeat(freqs[1],len(x2))

        ax.plot(x1,y1/1e6,color=colors[j])
        ax.plot(x2,y2/1e6,color=colors[j])
        ax.plot([x1[-1],x1[-1]],[0,y1[0]/1e6],color=colors[j],label=survey_all[j]['name'])
        ax.plot([x2[-1],x2[-1]],[0,y2[0]/1e6],color=colors[j])

        ax.fill_between(x,y/1e6,color=colors[j],alpha=0.4)
        ax.fill_between(x2,y2/1e6,y1[0:len(x2)]/1e6,color=colors[j],alpha=0.4)

    ax.legend(fontsize=fs,loc=(0.08,0.7))
    ax.set_xticks(np.arange(0.1,1.1,0.1))
    ax.tick_params(labelsize=fs,axis='both')
    ax.set_xlim(0,1)
    ax.set_ylim(200,1800)
    ax.set_xlabel('$\lambda^2$ (m$^2$)',fontsize=fs)
    ax.set_ylabel(r'$\nu$ (MHz)',fontsize=fs)

    # Make the RMSF plot:

    insetPosition = [0.395, 0.4, 0.49, 0.45]
    ax2 = fig.add_axes(insetPosition)
    for j in range(0,len(freqs_all)):
        ax2.plot(phi_all[j], rmsf_all[j],color=colors[j],linewidth=2)
        ax2.plot([-phi_broad_all[j]/2,-phi_broad_all[j]/2],[0,1.1],color=colors[j],
                 linestyle='dashed',linewidth = 2)
        ax2.plot([ phi_broad_all[j]/2, phi_broad_all[j]/2],[0,1.1],color=colors[j],
                 linestyle='dashed',linewidth = 2)
    if do_combo:
        ax2.plot(phi_combo, rmsf_combo,color='k',linewidth=2)
        ax2.plot([-phi_broad_combo/2,-phi_broad_combo/2],[0,1.1],color='k',
                 linestyle='dashed',linewidth = 2)
        ax2.plot([ phi_broad_combo/2, phi_broad_combo/2],[0,1.1],color='k',
                 linestyle='dashed',linewidth = 2)
    ax2.set_xticks(np.arange(-300,400,100))
    ax2.tick_params(labelsize=fs,axis='both')
    ax2.set_xlim(-phi_max,phi_max)
    ax2.set_ylim(0,1.1)
    ax2.set_xlabel('Faraday Depth (rad m$^{-2}$)',fontsize=fs)
    ax2.set_ylabel('RMSF',fontsize=fs)

    plt.savefig(file_out)

    return

## Calculate RMSFs

In [ ]:
freqs_all     = []
phi_broad_all = []
phi_all       = []
rmsf_all      = []

for survey in survey_all:
    
    fmin = survey['fmin']
    fmax = survey['fmax']
    df = survey['df']
    RMSFout = survey['file']

    cmd = 'python3 calculate_RMSF_AO.py -m '+str(phi_max)+' -d '+str(dphi)+' -f '+str(fmin)+' '+str(fmax)+' '+str(df)+' -o '+RMSFout
    !{cmd}
    
    freqs_all.append([survey['fmin'],survey['fmax']])
    phi, rmsf, phi_broad = get_survey_params(survey)
    phi_broad_all.append(phi_broad)
    phi_all.append(phi)
    rmsf_all.append(rmsf)
    
if combine:
    fcombine = []
    for survey in survey_all:
        fcombine.extend(list(np.arange(survey['fmin'],survey['fmax']+survey['df'],survey['df'])))
    with open('combo_freq.txt', 'w') as filehandle:
        for i in range(0,len(fcombine)):
            filehandle.write(f"{fcombine[i]}\n")
    cmd = 'python3 calculate_RMSF_AO.py combo_freq.txt -m '+str(phi_max)+' -d '+str(dphi)+' -o combo'
    !{cmd}
    phi_combo, rmsf_combo, phi_broad_combo = get_combo_survey_params()
    
cmd = 'rm *.txt'
!{cmd}

## Make the plot

In [ ]:
make_the_plot(survey_all,colors,freqs_all,phi_broad_all,phi_all,rmsf_all,fs=24,
              phi_combo=phi_combo, rmsf_combo=rmsf_combo, phi_broad_combo=phi_broad_combo, 
              do_combo=combine,file_out=file_out)